# Домашнее задание по теме «Многослойная нейронная сеть»

**Цель задания:** научиться на простейшем уровне подбирать архитектуру полносвязной нейронной сети для получения качества решения задачи не ниже заданного.

# Задание

Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (datasets).
Получите качество на тестовой выборке не ниже 88%

In [ ]:
#импорт пакетов для работы
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision as tv
import time

In [ ]:
#создаем тренировочный и тестовый наборы данных
BATCH_SIZE=256

train_dataset = tv.datasets.MNIST('F_MNIST_data', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.MNIST('F_MNIST_data', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [ ]:
#посмотрим на тренировочный сет
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: F_MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
#посмотрим на тестовый сет
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: F_MNIST_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
#создадим простую архитектуру модели из 4 слоев
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

#посмотрим на результат
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
#зададим дополнительные параметры для обучения
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.SGD(model.parameters(), lr=.01)
num_epochs = 10

In [ ]:
#опишем функию обучения
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()

        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)

        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)

        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [ ]:
#обучим модель
train_model()

ep: 0, taked: 9.933, train_loss: 2.027061049988929, train_acc: 0.6004666666666667, test_loss: 1.6700386911630631, test_acc: 0.7525
ep: 1, taked: 9.870, train_loss: 1.3201248825864589, train_acc: 0.7859833333333334, test_loss: 0.9937987864017487, test_acc: 0.8212
ep: 2, taked: 9.017, train_loss: 0.8535227657632625, train_acc: 0.8301166666666666, test_loss: 0.7032222546637058, test_acc: 0.8488
ep: 3, taked: 9.934, train_loss: 0.6565881427298201, train_acc: 0.8516833333333333, test_loss: 0.572273439541459, test_acc: 0.8666
ep: 4, taked: 9.889, train_loss: 0.5580019723861775, train_acc: 0.86445, test_loss: 0.4993649713695049, test_acc: 0.8771
ep: 5, taked: 9.990, train_loss: 0.49919871618138983, train_acc: 0.87335, test_loss: 0.453103581815958, test_acc: 0.8844
ep: 6, taked: 8.985, train_loss: 0.4601395341944187, train_acc: 0.8799833333333333, test_loss: 0.4212162496522069, test_acc: 0.8901
ep: 7, taked: 9.867, train_loss: 0.43225716789986224, train_acc: 0.88485, test_loss: 0.3979058826342